In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from numpy import array
import matplotlib.lines as mlines
from itertools import islice
import csv
import pandas as pd
import math
import glob
import re

## Obtain DT, thickness and error from the raw data csv files


In [ ]:
def getData(inputpath, outputpath, outfilename):
  
    files = glob.glob(inputpath+'*csv')
    cols1 = ['number', 'time', 'T0', 'T1','T2', 'T3', 'T4', 'T5']
    cols2 = ['number', 'time','T3', 'T4', 'T5']
    cols3 = ['number', 'time','T0', 'T2', 'T3', 'T4','T5']
    cols4 = ['number', 'time','T4', 'T3', 'T5']

    x = []
    y = []
    y_errs = []
    for i, file in enumerate(files):
        print(file)

        # read file
        df = pd.read_csv(file, header=None, names=cols3)
        df_sub = df[['number', 'T3', 'T4', 'T5']].iloc[ 7:] # take only the last 20 min data
        df_sub = df_sub.astype('float')

        DT1 = np.array(df_sub['T3'] - df_sub['T4'])
        DT2 = np.array(df_sub['T4'] - df_sub['T5'])
        timestamp = np.array(df_sub['number'])

        # plot DT vs timestamp

        fig = plt.figure(figsize=(11,8))

        plt.plot(timestamp, DT1, '-b', label='T3 - T4')

        plt.annotate(DT1[-1],(timestamp[-1], DT1[-1]), textcoords="offset points", xytext=(0,10), ha='center')
        plt.annotate(DT2[-1],(timestamp[-1], DT2[-1]), textcoords="offset points", xytext=(0,10), ha='center')

        plt.plot(timestamp, DT2, '-r', label='T4 - T5')

        plt.xlabel('Timestamp')
        plt.ylabel('DeltaT')
        plt.title(file.split('/')[-1])
        plt.ylim(-1,10)
        plt.xlim(0,5000)
        plt.legend()
        # plt.show()
        fig.savefig('{}.png'.format(inputpath + file.split('/')[-1]))

        # obtain data

        DT1 = DT1[(len(df) - 1200) : ]
        DT2 = DT2[(len(df) - 1200) : ]

        y.append(DT1.mean()) 
        y.append(DT2.mean())
        y_errs.append(DT1.std())
        y_errs.append(DT2.std())

        thick = re.findall('\d+', (file.split('/')[-1]).split('_')[1]) # get sample thickness from the file name, should be aware of the sample order
        x.append(thick[0])
        x.append(thick[1])

    x = np.array(x)
    x = x.astype('float')
    
    # write to a csv file
    df = pd.DataFrame({"Thickness" : x, "DeltaT" : y, "Error" : y_errs})
    df.to_csv(outputpath+outfilename+".csv", index=False)


In [ ]:
inpath = "/Users/wei/Documents/Max_lab/AlCF_2020_2/"
outpath = '/Users/wei/Desktop/'

getData(inputpath = inpath, outputpath = outpath, outfilename = 'output')